In [1]:
%load_ext autoreload
%autoreload 2

import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [2]:
from main import train

faster_rcnn = train()

Epoch 0: 100%|██████████| 313/313 [01:55<00:00,  2.70it/s, loss=66.7, rpn_cls_loss=0.168, rpn_reg_loss=1.32, rcnn_cls_loss=0.494, rcnn_reg_loss=5.28, accuracy=38.8, offsets=5.75]


In [5]:
from lib.faster_rcnn_profiling import profiler as rcnn_profiler
from lib.trainer_efficient import profiler as trainer_profiler

print(rcnn_profiler.stats)

print(trainer_profiler.stats)

defaultdict(<class 'int'>, {'forward_backbone': 18.05685783700619, 'forward_rpn': 3.1292235049963892, 'forward_nms': 5.323971187001462, 'forward_roi_pooling': 3.0274383310011217, 'forward_rcnn': 9.694891438000468})
defaultdict(<class 'int'>, {'forward_rpn_trainer_match_target_to_anchor': 0.34172457700333325, 'forward_rpn_trainer': 0.6019632110014754, 'forward_rcnn_trainer': 14.1993548260034})


In [ ]:
from torch.nn import functional as F
import torchvision.transforms as transforms
from torch import autograd, nn, optim, utils

from dataset_test import Dataset, color_map
from lib.predictor_efficient import FasterRCNNPredictor

dataset = Dataset(transforms.Compose([transforms.ToTensor()]), min_bboxes=3, max_bboxes=7)
dataloader = utils.data.DataLoader(dataset, batch_size=4, num_workers=12, shuffle=True)

dataloader_iter = iter(dataloader)

In [ ]:
faster_rcnn_predictor = FasterRCNNPredictor(faster_rcnn).cuda()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

import torch

from helper import visualize_anchors, visualize_rpn, visualize_rcnn, img_to_np

img, targets, classes = next(dataloader_iter)
img = img.float().cuda()
targets = targets.detach().float().cuda()
img_id = torch.arange(0, img.size()[0])

nms_reg_rounded, nms_reg, nms_cls, rcnn_reg, rcnn_cls, reg, cls = faster_rcnn_predictor(img)

i = 1

fig, ax = plt.subplots(figsize=(10, 10))
np_img = img_to_np(img[i])
ax.imshow(np_img)
visualize_rpn(ax, nms_reg.cpu()[i], nms_cls.cpu()[i], img.cpu()[i], [1.0, 1.0, 1.0], draw_all=False)
visualize_rpn(ax, nms_reg_rounded.cpu()[i], nms_cls.cpu()[i], img.cpu()[i], [1.0, 0, 0], draw_all=False)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
np_img = img_to_np(img[i])
ax.imshow(np_img)
visualize_rcnn(ax, reg[i].cpu(), cls[i].cpu(), color_map)
plt.show()